In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import cross_val_score,GridSearchCV

from sklearn import ensemble
import xgboost

import warnings
warnings.filterwarnings('ignore')

class_col='Survived'
id_col='PassengerId'

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os 
print(os.listdir("../data")) # this will list files in input directory

# Any results you write to the current directory are saved as output.

FileNotFoundError: [Errno 2] No such file or directory: '../data'

# Load Data

In [2]:
raw_train=pd.read_csv('../data/train.csv')
raw_test=pd.read_csv('../data/test.csv')

In [3]:
raw_train.info() # So, there are some null in Age, Cabin and Embarked

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
raw_train.tail(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [5]:
raw_test.info() # nulls in Age, Cabin and embarked ( same as train data), here in Fare Also

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [6]:
raw_test.tail(1)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


We are merging data together, so we can process data, after feature engineering we will split 

In [7]:
data=pd.concat([raw_train, raw_test], axis=0).reset_index(drop=True) # concatinating test and train removing index
data.head(2)
data.tail(2) # surviced will be NaN for test data

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
1307,NaN,NaN,S,8.0500,"Ware, Mr. Frederick",0,1308,3,male,0,NaN,359309
1308,NaN,NaN,C,22.3583,"Peter, Master. Michael J",1,1309,3,male,1,NaN,2668


In [8]:
# nulls in 4 columns -> Age, Embarked, cabin, Fare ( Survived will not count as it will not have value for test data)
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [9]:
data.isnull().sum()  # finding null per column

Age             263
Cabin          1014
Embarked          2
Fare              1
Name              0
Parch             0
PassengerId       0
Pclass            0
Sex               0
SibSp             0
Survived        418
Ticket            0
dtype: int64

# Visualization & Data Cleaning

# Modeling

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 16 columns):
Age            1309 non-null float64
Cabin          1309 non-null object
Embarked       1309 non-null object
Fare           1309 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
Title          1309 non-null object
Fsize          1309 non-null int64
FamilyLabel    1309 non-null int64
TicketGroup    1309 non-null int64
dtypes: float64(3), int64(7), object(6)
memory usage: 163.8+ KB


In [44]:
data.head(1)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title,Fsize,FamilyLabel,TicketGroup
0,22.0,X,S,7.25,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,Mr,2,2,1


In [45]:
#choosing required columns and converting to 0/1 for strings
data=data[['Survived','Age','Cabin','Embarked','Fare','Pclass','Sex','TicketGroup','FamilyLabel','Title']]
data=pd.get_dummies(data)
data.head()

,Survived,Age,Fare,Pclass,TicketGroup,FamilyLabel,Cabin_A,Cabin_B,Cabin_C,Cabin_D,...,Embarked_Q,Embarked_S,Sex_female,Sex_male,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
0,0.0,22.0,7.2500,3,1,2,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
1,1.0,38.0,71.2833,1,2,2,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
2,1.0,26.0,7.9250,3,1,1,0,0,0,0,...,0,1,1,0,0,1,0,0,0,0
3,1.0,35.0,53.1000,1,2,2,0,0,1,0,...,0,1,1,0,0,0,0,1,0,0
4,0.0,35.0,8.0500,3,1,1,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0


In [46]:
data.tail()

,Survived,Age,Fare,Pclass,TicketGroup,FamilyLabel,Cabin_A,Cabin_B,Cabin_C,Cabin_D,...,Embarked_Q,Embarked_S,Sex_female,Sex_male,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
1304,NaN,21.0,8.0500,3,1,1,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
1305,NaN,39.0,108.9000,1,2,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
1306,NaN,38.5,7.2500,3,1,1,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
1307,NaN,21.0,8.0500,3,1,1,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
1308,NaN,9.0,22.3583,3,2,2,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 26 columns):
Survived         891 non-null float64
Age              1309 non-null float64
Fare             1309 non-null float64
Pclass           1309 non-null int64
TicketGroup      1309 non-null int64
FamilyLabel      1309 non-null int64
Cabin_A          1309 non-null uint8
Cabin_B          1309 non-null uint8
Cabin_C          1309 non-null uint8
Cabin_D          1309 non-null uint8
Cabin_E          1309 non-null uint8
Cabin_F          1309 non-null uint8
Cabin_G          1309 non-null uint8
Cabin_T          1309 non-null uint8
Cabin_X          1309 non-null uint8
Embarked_C       1309 non-null uint8
Embarked_Q       1309 non-null uint8
Embarked_S       1309 non-null uint8
Sex_female       1309 non-null uint8
Sex_male         1309 non-null uint8
Title_Master     1309 non-null uint8
Title_Miss       1309 non-null uint8
Title_Mr         1309 non-null uint8
Title_Mrs        1309 non-null uint8


In [52]:
train=data[:len(raw_train)] # take train data
test=data[len(raw_train):].drop(['Survived'],axis=1) # take test data
X = train.drop(['Survived'],axis=1)
y = train.Survived

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [61]:
model = ensemble.RandomForestClassifier(random_state = 5, 
                                      warm_start = True,
                                      n_estimators = 26, 
                                      max_depth = 6, 
                                      max_features = 'sqrt')
clf = model.fit(X_train,y_train)
clf.score(X_test,y_test)

0.84593837535014

In [65]:
predictions = model.predict(test)
submission = pd.DataFrame({id_col: raw_test[id_col],
                           class_col: predictions.astype(np.int32)})
submission.to_csv("../output/output.csv", index=False)
